<a href="https://colab.research.google.com/github/Aruvi-B/Machine-Learning/blob/main/SPAM_SMS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("/content/spam.csv",encoding='latin-1')
data.head()

# Link: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [ ]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"label", "v2":"text"})

In [ ]:
data.describe()

,label,text
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [ ]:
data.groupby("label").describe()

text                                                               
      count unique                                                top freq
label                                                                     
ham    4825   4516                             Sorry, I'll call later   30
spam    747    653  Please call our customer service representativ...    4

In [ ]:
data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [ ]:
data['spam'] = data['label'].map( {'spam': 1, 'ham': 0} ).astype(int)
data.head()

,label,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
data['length'] = data['text'].apply(len)



In [ ]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def remove_punctuation_and_stopwords(text):
    text = "".join([char for char in text if char not in string.punctuation])
    text = [word.lower() for word in text.split() if word.lower() not in stopwords.words("english")]
    return " ".join(text)

data['text'] = data['text'].apply(remove_punctuation_and_stopwords)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install scikit-learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

bow_transformer = CountVectorizer().fit(data['text'])
bow_data = bow_transformer.transform(data['text'])

tfidf_transformer = TfidfTransformer().fit(bow_data)
data_tfidf = tfidf_transformer.transform(bow_data)


In [ ]:
from scipy.sparse import  hstack
from sklearn.model_selection import train_test_split
X2 = hstack((data_tfidf ,np.array(data['length'])[:,None])).A
X_train, X_test, y_train, y_test = train_test_split(X2, data["spam"], train_size=0.8, random_state = 42)

# Refer my Github = https://github.com/Aruvi-B/Machine-Learning/blob/main/Wine_Recognition_Six_algorithms.ipynb

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train.values.ravel())
rf_predition = rf_model.predict(X_test_scaled)

In [ ]:
# SVM
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train.values.ravel())
svm_predition = svm_model.predict(X_test_scaled)

In [ ]:
# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_scaled, y_train.values.ravel())
gb_predition = gb_model.predict(X_test_scaled)

In [ ]:
import seaborn as sns
models = ['Random Forest', "SVM", "Gradient Boost"]
predictions = [rf_predition, svm_predition, gb_predition]

for model, pred in zip(models, predictions):
    accuracy = accuracy_score(y_test, pred)
    conf_matrix = confusion_matrix(y_test, pred)
    classification_rep = classification_report(y_test, pred)

    print(f"Model: {model}")
    print(f"Accuracy: {accuracy}")
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", classification_rep)

    print("__________________________________________________________________")


Model: Random Forest
Accuracy: 0.9730941704035875
Confusion Matrix:
 [[965   0]
 [ 30 120]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       1.00      0.80      0.89       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

__________________________________________________________________
Model: SVM
Accuracy: 0.9695067264573991
Confusion Matrix:
 [[964   1]
 [ 33 117]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       0.99      0.78      0.87       150

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

_________________________________________________________